In [ ]:
import numpy as np
from keras import models,layers
import cv2
import pickle
import os 
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input,AveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Dir = "/Users/HP/Desktop/Coding/AIML/Face_Mask_Detection/Face_Mask_Dataset/Train"
Val_Dir = "/Users/HP/Desktop/Coding/AIML/Face_Mask_Detection/Face_Mask_Dataset/Validation"
categories = ['WithMask','WithoutMask']

INIT_LR = 1e-4
EPOCHS = 20
BS = 32

data = []
labels = []

val_data = []
val_labels =[]

for i in categories:
  path = os.path.join(Dir,i)
  for j in os.listdir(path):
    img_path = os.path.join(path,j)
    img = load_img(img_path,target_size=(224,224))
    img_arr = img_to_array(img)
    img_arr = preprocess_input(img_arr)

    data.append(img_arr)
    labels.append(i)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data,dtype="float32")
labels = np.array(labels)

data_aug = ImageDataGenerator(
    rotation_range= 22,
    zoom_range = 0.16,
    width_shift_range=0.2,
    height_shift_range =0.2,
    shear_range=0.20,
    horizontal_flip=True,
    fill_mode="nearest"
)

base_model = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape = (224,224,3)))

head = base_model.output
head = AveragePooling2D(pool_size=(7,7))(head)

head = keras.layers.Flatten(name="flatten")(head)
head = keras.layers.Dense(128,activation="relu")(head)
head = keras.layers.Dropout(0.5)(head)
head = keras.layers.Dense(2,activation = "softmax")(head)

model = keras.models.Model(inputs = base_model.input,outputs = head )

for i in base_model.layers:
  i.trainable =False

output = Adam(learning_rate = INIT_LR,decay =INIT_LR/EPOCHS)

model.compile(loss = "binary_crossentropy",optimizer = output,metrics=['accuracy'])

(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=38)

steps_per_epoch = len(train_x) // BS
validation_steps = len(test_x) // BS

history = model.fit(
    data_aug.flow(train_x, train_y ,batch_size=BS),
    steps_per_epoch=steps_per_epoch,
    validation_data=(test_x, test_y),
    validation_steps=validation_steps,
    epochs=EPOCHS)


In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []
    predictions = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face_detect = frame[startY:endY, startX:endX]
            face_detect = cv2.cvtColor(face_detect, cv2.COLOR_BGR2RGB)
            face_detect = cv2.resize(face_detect, (224, 224))
            face_detect = img_to_array(face_detect)
            face_detect = preprocess_input(face_detect)

            faces.append(face_detect)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        predictions = maskNet.predict(faces, batch_size=32)

    return (locs, predictions)

prototxtPath = r"Face_detector\deploy.prototxt"
weightsPath = r"Face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("mask_detector_new.model")

vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    (locs, predictions) = detect_and_predict_mask(frame, faceNet, maskNet)

    for (box, pred) in zip(locs, predictions):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(frame, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()
